# Create Metadata entries 

In [49]:
from metacatalog import api
import pandas as pd
from pyproj import Transformer

Create the database session

In [2]:
session = api.connect_database()

In [20]:
raw = pd.read_excel('Buhlot_raw.xlsx')
print(raw.shape)
raw.head(26)

(26, 15)


,ID_V4W,Name,Operator,ID_Buhlot,Area,"Easting [GK, m]","Northing [GK, m]",m.a.s.l [m],Variable Type,Units,Symbol,Unit_ID,Keyword_ID,Measuring device,Comments
0,A_1,Altschweier / Bühlot LUBW,LUBW,28.19.00_00_01,30.18,3437511.000,5395848.000,141.58,Discharge,[m³/s],Q,108,7327.0,Drucksonde Endress und Hauser Waterpilot FMX 165,Data 2013-2019 NaN free
1,A_2,Bühlertal Kindergarten / Bühlot,HZV,28.13.00_00_01,12.79,3440860.017,5393606.371,294.00,Discharge,[m³/s],Q,108,7327.0,Radarpegel Endress und Hauser Micropilot FMR 230V,Data 2013-2019 / 32% of NaN = 19914 NaN / HZV ...
2,A_3,Schwabenbrünnele / KIT,KIT,28.11.03.01_00_01,0.12,3442007.862,5389779.800,964.00,Water level,[cm above reference datum],L,2,NaN,OTT CTD,Data 2013-2019 / 30% of NaN = 18459 NaN / Tras...
3,A_4_1,baiersbronn-ruhestein,DWD,NaN,nn,3442700.000,5380700.000,916.00,Precipitation,[mm],P,103,6434.0,nn,Data 2013-2019 / 0.2% of NaN = 134 NaN
4,A_5_1,buehl_agramet,Agramet,NaN,nn,3435792.645,5392646.888,162.00,Precipitation,[mm],P,103,6434.0,nn,Data 2013-2019 / 2.4% of NaN = 1514 NaN
5,A_6,butschenberg,KIT,28.14.04_00_01,nn,3441694.515,5394526.540,379.50,Precipitation,[mm],P,103,6434.0,Davis Tipping Bucket Rain Collektor,Data 2013-2019 / 25% of NaN = 15090 NaN
6,A_7,p_hrb20,HZV,NaN,nn,3435735.697,5398043.979,133.50,Precipitation,[mm],P,103,6434.0,nn,Data with 38% of NaN = 23552 NaN
7,A_8,bühlertal-kindergarten,HZV,28.11.00_00_02,nn,3440841.497,5393606.635,303.00,Precipitation,[mm],P,103,6434.0,Thies-Clima 5.4032.45.008 mit Heizung,Data 2013-(Jul)2015 / 38% of NaN = 23444 NaN
8,A_9,neuweier_agramet,Agramet,NaN,nn,3439398.068,5398056.333,210.00,Precipitation,[mm],P,103,6434.0,nn,Data 2013-2019 / 0.02% of NaN = 18 NaN
9,A_10,Altschweier Ortseingang / Bühlot,HZV,28.19.00_00_03,27.88,3439171.867,5395745.108,183.00,Precipitation,[mm],P,103,6434.0,nn,Data 2013-2019 / 38% of NaN = 23631 NaN / HZV ...


## Creating owner

In [5]:
raw.Operator.unique()

array(['LUBW', 'HZV', 'KIT', 'DWD', 'Agramet'], dtype=object)

In [6]:
# check if existing
lubw = api.find_person(session, organisation_name='Landesanstalt für Umwelt Baden-Württemberg', return_iterator=True).first()
if lubw is None:
    lubw = api.add_person(session, first_name='Uwe', last_name='Ehret', 
                          organisation_name="Landesanstalt für Umwelt Baden-Württemberg",
                          affiliation="LUBW Landesanstalt für Umwelt Baden-Württemberg, Karlsruhe, Germany",
                         citation="Grundlage: Daten aus dem Umweltinformationssystem (UIS) der LUBW Landesanstalt für Umwelt Baden-Württemberg")
#hzv = api.find_person(session, last_name='HZV', return_iterator=True).first()
#if hzv is None:
#    hzv = api.add_person(session, first_name=None, last_name='HZV', organisation_name="Hochwasserzweckverband")
kit = api.find_person(session, last_name='Ehret', return_iterator=True).first()
if kit is None:
    kit = api.add_person(session, first_name='Uwe', last_name='Ehret', 
                         organisation_name='Karlsruhe Institute of Technology (KIT)', 
                         affiliation='Institute of Water and River Basin Management, Karlsruhe Institute of Technology, Germany.'
                        )
#dwd = api.find_person(session, last_name='DWD', return_iterator=True).first()
#if dwd is None:
#    dwd = api.add_person(session, last_name='DWD', organisation_name='Deutscher Wetterdienst')
#agramet = api.find_person(session, last_name='Agramet', return_iterator=True).first()
#if agramet is None:
#    agramet = api.add_person(session, last_name='Agramet', organisation_name='Agramet')

In [7]:
print(kit)

Uwe Ehret <ID=3>


## Variables

In [9]:
raw['Variable Type'].unique()

array(['Discharge', 'Water level', 'Precipitation', 'Temperature',
       'Relative Humidity', 'Global Radiation / Solar Irradiance',
       'Wind Speed', 'Snow Water Equivalent', 'Evapotranspiration',
       'Soil Moisture TETA', 'Ground Water Level'], dtype=object)

In [19]:
raw.where(raw.Operator=='KIT')['Variable Type'].unique()

array([nan, 'Water level', 'Precipitation', 'Temperature',
       'Soil Moisture TETA', 'Ground Water Level'], dtype=object)

In [14]:
variables = api.find_variable(session)
for v in variables:
    print(v)

air temperature [C] <ID=1>
soil temperature [C] <ID=2>
water temperature [C] <ID=3>
discharge [m3/s] <ID=4>
air pressure [10^2*Pa] <ID=5>
relative humidity [%] <ID=6>
daily rainfall sum [mm/d] <ID=7>
rainfall intensity [mm/h] <ID=8>
solar irradiance [W/m2] <ID=9>
net radiation [W/m2] <ID=10>
gravimetric water content [kg/kg] <ID=11>
volumetric water content [cm3/cm3] <ID=12>
precision [-] <ID=13>
sap flow [cm^3/cm^2h] <ID=14>
matric potential [MPa] <ID=15>
bulk electrical conductivity [EC] <ID=16>
specific electrical conductivity [EC] <ID=17>
river water level [m] <ID=18>


In [16]:
for u in api.find_unit(session):
    print(u)

second <ID=1>
meter <ID=2>
kilogram <ID=3>
ampere <ID=4>
kelvin <ID=5>
mole <ID=6>
candela <ID=7>
radian <ID=8>
degree <ID=9>
hertz <ID=10>
newton <ID=11>
pascal <ID=12>
joule <ID=13>
watt <ID=14>
coulomb <ID=15>
volt <ID=16>
farad <ID=17>
ohm <ID=18>
siemens <ID=19>
lux <ID=20>
relative <ID=21>
mass flux density per hour <ID=22>
hour <ID=23>
megapascal <ID=24>
electrical conductivity <ID=25>
degree Celsius <ID=101>
milimeter <ID=102>
mm per day <ID=103>
hectopascal <ID=104>
mm per hour <ID=105>
mm per second <ID=106>
meter per second <ID=107>
cubicmeter per second <ID=108>
liter per second <ID=109>
degree <ID=110>
percent <ID=112>
cm3/cm3 <ID=113>
kg/kg <ID=114>
watt per sqauaremeter <ID=115>


In [30]:
# missing variables:
wind = api.find_variable(session, name='wind speed', return_iterator=True).first()
if wind is None:
    wind = api.add_variable(session, name='wind speed', symbol='v', unit=107)
ground = api.find_variable(session, name='ground water level', return_iterator=True).first()
if ground is None:
    ground = api.add_variable(session, name='ground water level', symbol='GWL', unit=2)
evapo = api.find_variable(session, name='evapotranspiration', return_iterator=True).first()
if evapo is None:
    evapo = api.add_variable(session, name='evapotranspiration', symbol='ET', unit=105)
we = api.find_variable(session, name='water equivalent', return_iterator=True).first()
if we is None:
    we = api.add_variable(session, name='water equivalent', symbol='WE', unit=102)
    
# variable mapping
v_mapping = {
    'Discharge': 4,
    'Water level': 18,
    'Precipitation': 8,
    'Relative Humidity': 6,
    'Global Radiation / Solar Irradiance': 9,
    'Wind Speed': wind.id,
    'Soil Moisture TETA': 12,
    'Ground Water Level': ground.id,
    'Temperature': 1,
    'Snow Water Equivalent': we.id,
    'Evapotranspiration': evapo.id
}



In [32]:
variable_ids = raw['Variable Type'].map(lambda v: v_mapping[v])

### Notes

* GWL in mm? -> transform to m
* water level in cm? -> transform to m
* rainfall unit ? 

## Location

In [50]:
t = Transformer.from_crs('epsg:31467', 'epsg:4326')

In [87]:
locations = ['SRID=4326;POINT (%f %f)' % t.transform(r[0], r[1]) for i, r in raw[['Easting [GK, m]', 'Northing [GK, m]']].iterrows()]

## License

In [65]:
lic = api.find_license(session, short_title='CC BY %', return_iterator=True ).first()
print(lic)

Creative Commons Attribution 4.0 International <ID=6>


## Metadata

In [88]:
# only use the KIT operator for now
df = raw.copy()
#df = raw
df['location'] = locations
df['variable_id'] = variable_ids
df['license_id'] = lic.id
df['title'] = df.Name.map(lambda n: 'Bühlot dataset: %s' % n)

df.head()

,ID_V4W,Name,Operator,ID_Buhlot,Area,"Easting [GK, m]","Northing [GK, m]",m.a.s.l [m],Variable Type,Units,Symbol,Unit_ID,Keyword_ID,Measuring device,Comments,location,variable_id,license_id,title
0,A_1,Altschweier / Bühlot LUBW,LUBW,28.19.00_00_01,30.18,3437511.000,5395848.000,141.58,Discharge,[m³/s],Q,108,7327.0,Drucksonde Endress und Hauser Waterpilot FMX 165,Data 2013-2019 NaN free,SRID=4326;POINT (29.598817 28.380061),4,6,Bühlot dataset: Altschweier / Bühlot LUBW
1,A_2,Bühlertal Kindergarten / Bühlot,HZV,28.13.00_00_01,12.79,3440860.017,5393606.371,294.00,Discharge,[m³/s],Q,108,7327.0,Radarpegel Endress und Hauser Micropilot FMR 230V,Data 2013-2019 / 32% of NaN = 19914 NaN / HZV ...,SRID=4326;POINT (29.630629 28.363901),4,6,Bühlot dataset: Bühlertal Kindergarten / Bühlot
2,A_3,Schwabenbrünnele / KIT,KIT,28.11.03.01_00_01,0.12,3442007.862,5389779.800,964.00,Water level,[cm above reference datum],L,2,NaN,OTT CTD,Data 2013-2019 / 30% of NaN = 18459 NaN / Tras...,SRID=4326;POINT (29.646054 28.328567),18,6,Bühlot dataset: Schwabenbrünnele / KIT
3,A_4_1,baiersbronn-ruhestein,DWD,NaN,nn,3442700.000,5380700.000,916.00,Precipitation,[mm],P,103,6434.0,nn,Data 2013-2019 / 0.2% of NaN = 134 NaN,SRID=4326;POINT (29.665335 28.241237),8,6,Bühlot dataset: baiersbronn-ruhestein
4,A_5_1,buehl_agramet,Agramet,NaN,nn,3435792.645,5392646.888,162.00,Precipitation,[mm],P,103,6434.0,nn,Data 2013-2019 / 2.4% of NaN = 1514 NaN,SRID=4326;POINT (29.588925 28.345989),8,6,Bühlot dataset: buehl_agramet


In [89]:
df = df.where(df.Operator=='KIT').dropna()
df.drop(['ID_V4W', 'Name', 'Operator', 'Easting [GK, m]', 'Northing [GK, m]', 'Variable Type', 'Units', 'Symbol', 'Unit_ID', 'Keyword_ID'], axis=1, inplace=True)
df.head()

,ID_Buhlot,Area,m.a.s.l [m],Measuring device,Comments,location,variable_id,license_id,title
5,28.14.04_00_01,nn,379.5,Davis Tipping Bucket Rain Collektor,Data 2013-2019 / 25% of NaN = 15090 NaN,SRID=4326;POINT (29.636368 28.374277),8.0,6.0,Bühlot dataset: butschenberg
10,28.13.01_01_01,nn,474.0,nn,Data 2013-2019 / 21% of NaN = 13047 NaN,SRID=4326;POINT (29.630132 28.347483),8.0,6.0,Bühlot dataset: Schafhof
11,28.00.00_00_01,nn,646.5,Davis Tipping Bucket Rain Collektor,Data 2013-2019 / 34% of NaN = 20971 NaN,SRID=4326;POINT (29.627973 28.337095),8.0,6.0,Bühlot dataset: schönbrunn
12,28.11.03.01_00_02,nn,964.0,Davis Tipping Bucket Rain Collektor,Data 2013-2019 / 64% of NaN = 39307 NaN,SRID=4326;POINT (29.646054 28.328567),8.0,6.0,Bühlot dataset: schwabenbrünnele
13,28.15.01_00_01,nn,326.0,Davis Tipping Bucket Rain Collektor,Data 2013-2019 / 12% of NaN = 7596 NaN,SRID=4326;POINT (29.613377 28.372382),8.0,6.0,Bühlot dataset: sportplatz


In [90]:
col_map = {
    'ID_Buhlot': 'external_id',
    'm.a.s.l [m]': 'elevation',
    'Comments': 'comment'
}
df.columns = [col_map.get(col, col) for col in df.columns]
df['variable_id'] = df.variable_id.astype(int)
df['license_id'] = df.license_id.astype(int)
df.head()

,external_id,Area,elevation,Measuring device,comment,location,variable_id,license_id,title
5,28.14.04_00_01,nn,379.5,Davis Tipping Bucket Rain Collektor,Data 2013-2019 / 25% of NaN = 15090 NaN,SRID=4326;POINT (29.636368 28.374277),8,6,Bühlot dataset: butschenberg
10,28.13.01_01_01,nn,474.0,nn,Data 2013-2019 / 21% of NaN = 13047 NaN,SRID=4326;POINT (29.630132 28.347483),8,6,Bühlot dataset: Schafhof
11,28.00.00_00_01,nn,646.5,Davis Tipping Bucket Rain Collektor,Data 2013-2019 / 34% of NaN = 20971 NaN,SRID=4326;POINT (29.627973 28.337095),8,6,Bühlot dataset: schönbrunn
12,28.11.03.01_00_02,nn,964.0,Davis Tipping Bucket Rain Collektor,Data 2013-2019 / 64% of NaN = 39307 NaN,SRID=4326;POINT (29.646054 28.328567),8,6,Bühlot dataset: schwabenbrünnele
13,28.15.01_00_01,nn,326.0,Davis Tipping Bucket Rain Collektor,Data 2013-2019 / 12% of NaN = 7596 NaN,SRID=4326;POINT (29.613377 28.372382),8,6,Bühlot dataset: sportplatz


### Upload

In [91]:
if False:
    for i,e in df.iterrows():
        entry = api.add_entry(session, e.title, kit.id, e.location, e.variable_id, 
                              external_id=e.external_id, license=e.license_id, embargo=False)

        details = {k:v for k,v in e.to_dict().items() if k not in ['title', 'external_id', 'variable_id', 'license_id', 'location']}
        api.add_details_to_entries(session, [entry], **details)

In [93]:
entries = api.find_entry(session, author='Ehret')
for e in entries:
    print(e)

<ID=18 Bühlot dataset: buts [rainfall intensity] >
<ID=19 Bühlot dataset: Scha [rainfall intensity] >
<ID=20 Bühlot dataset: schö [rainfall intensity] >
<ID=21 Bühlot dataset: schw [rainfall intensity] >
<ID=22 Bühlot dataset: spor [rainfall intensity] >
<ID=23 Bühlot dataset: wint [rainfall intensity] >
<ID=24 Bühlot dataset: schw [air temperature] >
<ID=25 Bühlot dataset: Scha [volumetric water content] >
<ID=26 Bühlot dataset: Spre [ground water level] >
<ID=27 Bühlot dataset: Scha [ground water level] >


## create the group

In [95]:
buhlot = api.add_group(session, 'Project', 
                       entry_ids=[e.id for e in entries],
                       title='Bühlot Dataset',
                       description='The Bühlot dataset is a collection of environmental measurements from the Bühlot.'
                      )
print(buhlot)

Project Bühlot Dataset <ID=1>


In [97]:
buhlot.entries